In [1]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
import sqlite3
import os
import glob

sns.set()
con = sqlite3.connect("./data.db")

# con.execute(('CREATE TABLE IF NOT EXISTS "siam" ('
#              '"ln" INTEGER,'
#              '"epoch" INTEGER,'
#              '"val_loss" REAL,'
#              '"val_categorical_accuracy" REAL,'
#              '"loss" REAL,'
#              '"categorical_accuracy" REAL,'
#              '"use_batch_norm" TEXT,'
#              '"optimizer" TEXT,'
#              '"activation" TEXT,'
#              '"layers" TEXT,'
#              '"train_size" INTEGER,'
#              '"reduction_method" TEXT,'
#              '"number_of_feature" INTEGER'
#              ');'))
# con.execute('CREATE INDEX "RM" on "per" ("reduction_method");')
# con.execute('CREATE INDEX "NF" on "per" ("number_of_feature");')
con.enable_load_extension(True)
con.load_extension("./extension-functions")

if not os.path.exists("graphs"):
    os.makedirs("graphs")

In [3]:
csv_paths = glob.glob('runs_results/*/history.csv')

if len(csv_paths) > 0:
    con.execute('drop table siam')
    for file in csv_paths:
        df = pd.read_csv(file)
        df.to_sql('siam', con, if_exists='append')

/home/GTL/nsix/Python3/lib/python3.6/site-packages/pandas/core/generic.py:1534: UserWarning: The spaces in these column names will not be changed. In pandas versions < 0.14, spaces were converted to underscores.
  chunksize=chunksize, dtype=dtype)


In [ ]:
def generate_sql_z_query(z_name: str, filters: dict):
    where_close = ""
    first_key = True
    for k, lf in filters.items():
        if first_key:
            first_key = False
        else:
            where_close += "AND "
        where_close += "("
        first = True
        for v in lf:
            if not first:
                where_close += " OR "
            else:
                first = False
            where_close += str(k)+"='"+str(v)+"'"
        where_close += ") "
    query = "SELECT DISTINCT "+z_name+" FROM per"
    if where_close != "":
        query += " WHERE " + where_close
    return query


def generate_sql_xy_query(x_name: str, y_name: str, z_name: str, z_value, filters: dict, x_limit):
    where_close = z_name+" = '"+str(z_value)+"' "
    for k, lf in filters.items():
        where_close += "AND ("
        first = True
        for v in lf:
            if not first:
                where_close += " OR "
            else:
                first = False
            where_close += str(k)+"='"+str(v)+"'"
        where_close += ") "
    if x_limit > 0:
        where_close += "AND " + x_name + " < " + str(x_limit) + " "
    query = ("SELECT " + x_name + ","
             "min(" + y_name + ") as min,"
             "max(" + y_name + ") as max,"
             "median(" + y_name + ") as median,"
             "avg(" + y_name + ") as mean "
             "FROM per ")
    if where_close != "":
        query += "WHERE " + where_close
    query += "GROUP BY " + x_name
    return query


def consolidate_data_from_db(con, x_name: str, y_name: str, z_name: str, filters: dict, x_limit=-1):
    z_value_df = pd.read_sql_query(generate_sql_z_query(z_name, filters), con)
    cd = dict()
    for i in z_value_df.get(z_name):
        query = generate_sql_xy_query(x_name, y_name, z_name, i, filters, x_limit)
        print("Selecting data for", z_name, "=", i)
        # print(query)
        cd[i] = pd.read_sql_query(query, con)
    return cd


def generate_plot(con, x_arg_name, y_arg_name, z_arg_name, filters, save_to_file=True, x_limit=-1, x_name=None, y_name=None):
    if x_name is None:
        x_name = x_arg_name
    if y_name is None:
        y_name = y_arg_name
        
    cd = consolidate_data_from_db(con, x_arg_name, y_arg_name, z_arg_name, filters, x_limit)

    plt.figure()
    plt.axes()
    for a, d in cd.items():
        plt.plot(d.get(x_arg_name), d.get('median'), 
                 label=z_arg_name.replace('_', ' ') + ' = ' + str(a).replace('_', ' '))
        # plt.fill_between(d.get(x_arg_name), d.get('mean') - d.get('std'), d.get('mean') + d.get('std'), alpha=0.3)
        plt.fill_between(d.get(x_arg_name), d.get('min'), d.get('max'), alpha=0.3)
    if y_arg_name == "val_categorical_accuracy":
        plt.ylabel("Validation accuracy")
    else:
        plt.ylabel(y_name.replace('_',' '))
    plt.xlabel(x_name.replace('_',' '))
    plt.legend()
    # plt.title("Evolution of acuracy acording to the number of training examples")
    if save_to_file:
        algo_name = "all_"
        if "reduction_method" in filters.keys():
            algo_name = '_'.join(map(str, filters["reduction_method"])) + '_'
        layers = ""
        if "layers" in filters.keys():
            layers = '_' + '_'.join(map(lambda x: x.replace('(','').replace(')','').replace(',',''), filters["layers"]))
        plt.savefig("graphs/"+algo_name+x_name+"_"+y_name+"_"+z_arg_name+layers+".png")
    plt.show()

/home/GTL/nsix/Python3/lib/python3.6/site-packages/pandas/core/generic.py:1534: UserWarning: The spaces in these column names will not be changed. In pandas versions < 0.14, spaces were converted to underscores.
  chunksize=chunksize, dtype=dtype)
